In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install selenium

In [ ]:
import threading
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request

In [ ]:
driver = webdriver.Chrome('C:/chromedriver.exe')
driver.get("https://www.google.com/search?q=white+color+t-shirt&source=lnms&tbm=isch&sa=X&sqi=2&ved=2ahUKEwjniaLxo473AhW6IDQIHblpDNIQ_AUoAXoECAEQAw&biw=1920&bih=969&dpr=1")

SCROLL_PAUSE_TIME = 3
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
SAVE_FLAG = False
def timeout(limit_time):
    start = time.time()
    while True:
        if time.time() - start > limit_time or SAVE_FLAG:
            raise Exception('timeout. or image saved.')

while True: # scroll to load results
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        try:
            driver.find_element_by_css_selector(".mye4qd").click()
        except:
            break
    last_height = new_height

images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
count = 0
for image in images:
    SAVE_FLAG = False
    timer = threading.Thread(target=timeout, args=(10))
    try:
        image.click()
        time.sleep(0.1)
        timer.start()
        # copy&paste image's XPath
        imgUrl = driver.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img').get_attribute("src")
        urllib.request.urlretrieve(imgUrl, "./dataset_image/"+ "_{0:04}".format(count) + ".jpg") # image save path
        print('Save images : ', "./dataset_image/"+ "_{0:04}".format(count) + ".jpg")
        SAVE_FLAG = True
        count += 1
        if timer.is_alive():
            timer.join()
    except Exception as e:
        if timer.is_alive():
            timer.join()
        pass
print('driver end. Total images : ', count)
driver.close()